<left>
<table style="margin-top:0px; margin-left:0px;">
<tr>
  <td><img src="https://raw.githubusercontent.com/worm-portal/WORM-Figures/master/style/worm.png" alt="WORM" title="WORM" width=50/></td>
  <td><h1 style=font-size:30px>Lake Honnedaga Activity</h1><h2>Answer Sheet</h2></td>
</tr>
</table>
</left>

In case you need it, here is a link to [the WORM database table](https://tinyurl.com/wormdb). It can be helpful for looking up mineral names. Descriptions of columns and their units can be found in the [WORM database documentation](https://worm-portal.asu.edu/docs/database/). 

---

### Activity: Lime the lake!

Honnedaga Lake in upstate New York is home to brook trout, *Salvelinus fontinalis*.

<img src="https://www.fws.gov/sites/default/files/styles/banner_image_xl/public/banner_images/2021-07/natdiglib_32715_full.jpg" alt="brook trout" title="Brook Trout" width=100%/>

Acidification of the lake from acid rain is suspected to have caused a marked decline in the population of brook trout [since the late 1960s](http://fish.dnr.cornell.edu/trout/honnedaga.htm).

You are investigating possible methods for improving the water quality of the lake. One technique for mitigating lake acidification is through a process called *liming*. This is where crushed agricultural limestone (CaCO$_3$) is spread evenly across the pond or lake with a barge. A crew of 3 can dispense around 10 tons of limestone per hour.

Model the water-rock reaction between Honnedaga Lake water and limestone.
- What happens to the pH of the lake?
- Acid rain has led to an increased concentration of dissolved sulfate in the lake. What happens to the speciation of sulfate when the lake is limed?
- Why does the pH of the lake increase when limestone is added?
- Lake Honnedaga has a surface area of 3.33 km$^2$ and an average depth of 15 m. About how many moles of limestone would be needed to saturate the lake? How many hours would it take a barge to dispense that much limestone?

Hint: assume limestone is mainly composed of calcite.

(Honnedaga Lake data is adapted from the [USGS Honnedaga Lake Tributary dataset](https://nwis.waterdata.usgs.gov/usa/nwis/qwdata/?site_no=0134277112&agency_cd=USGS&inventory_output=0&rdb_inventory_output=value&TZoutput=0&pm_cd_compare=Greater%20than&radio_parm_cds=all_parm_cds&format=html_table&qw_attributes=expanded&qw_sample_wide=separated_wide&rdb_qw_attributes=0&date_format=YYYY-MM-DD&rdb_compression=file&submitted_form=brief_list))

---

In [1]:
import aqequil

In [2]:
ae = aqequil.AqEquil(exclude_organics=True)

Loading Water-Organic-Rock-Microbe (WORM) thermodynamic databases...
Excluding ['organic_aq', 'organic_cr'] from column 'category_1'in wrm_data_latest.csv
wrm_data_latest.csv is now set as the active thermodynamic database.
Element database elements.csv is active.
Solid solution database solid_solutions.csv is active.
LogK database wrm_data_logk.csv is active.
Excluding ['organic_aq', 'organic_cr'] from column 'category_1'in wrm_data_logk.csv
LogK_S database wrm_data_logk_s.csv is active.
Excluding ['organic_aq', 'organic_cr'] from column 'category_1'in wrm_data_logk_s.csv
Loading thermodynamic database into pyCHNOSZ...


In [3]:
speciation = ae.speciate(input_filename="honnedaga.csv",
                         alter_options=[["antigorite", "Suppress"]],
                         )

The input file column 'logfO2' will be used to set sample redox state. If a another column is desired, set it manually using the redox_flag parameter.
Getting wrm_data_latest.csv ready. This will take a moment...
No 'charge_balance_on' column found in input file. Defaulting to no charge balancing for all samples.
Using wrm_data_latest.csv to speciate honnedaga-2013-06-11-1501
Finished!


In [4]:
speciation.save("honn")

Saved as 'honn.speciation'


The notebook can be safely shut down or restarted and resumed from this point.

In [5]:
import aqequil
speciation = aqequil.load("honn")

Attempting to load honn.speciation ...
Loaded 'honn.speciation'


In [6]:
For = aqequil.Reactant(reactant_name="calcite",
                       amount_remaining=1, # moles of mineral to react per kg H2O
                       f_rate_law='Relative rate equation', # forward rate law
                       f_eq1=1, # dXi(n)/dXi (mol/mol), the mole fraction of this mineral in the rock
                       b_rate_law="Partial equilibrium", # backward rate law
                       )

# combine the mineral reactants in a list to repare the reaction
r = aqequil.Prepare_Reaction(reactants=[For])

In [7]:
speciation = aqequil.react(speciation, r)

Using wrm_data_latest.csv to react honnedaga.2013.06.11.1501


In [8]:
speciation.lookup('name')

Column(s) not found: ['name']


Sample
honnedaga-2013-06-11-1501


In [9]:
# get mass transfer data for a sample
m = speciation.mt("honnedaga-2013-06-11-1501")

## Activity Answers

---
### What happens to the pH of the lake?

In [10]:
m.plot_pH()

---
### Acid rain has led to an increased concentration of dissolved sulfate in the lake. What happens to the speciation of sulfate when the lake is limed?

- Ca from the calcite starts to form a complex with SO4-2.

In [11]:
m.plot_mass_contribution("SO4-2")

---
### Why does the pH of the lake increase when limestone is added?

The liming reaction is:

CaCO3(cr) + SO4-2 + 2H+ → CaSO4(aq) + CO2 + H2O

Protons are being removed from the water as CaSO4 and CO2 is formed.

---
### Lake Honnedaga has a surface area of 3.33 km 2 and an average depth of 15 m. About how many moles of limestone would be needed to saturate the lake? How many hours would it take a barge to dispense that much limestone?

First, plot moles of product minerals. Show reactants and set log to False.

Isolate calcite's trend.

Look at how many moles of calcite you start and end with before reaching saturation.

In [12]:
m.plot_product_minerals(show_reactant_minerals=True, log_y=False, y_type='mole')

In [13]:
# number of moles of calcite needed to saturate 1 kg of H2O
# calculated from looking at the starting and ending moles of calcite in this plot:
# m.plot_product_minerals(show_reactant_minerals=True, log_y=False, y_type='mole')
moles_calcite_per_kg_H2O = 1 - 0.99847

Now work through the rest of the math. Calculate the mass of water in the lake:

In [14]:
# lake area in m2 = 3.33 km2 * 1000000 m2/km2
m2 = 3.33 * 1000000

# lake volume in m3 = m2 * 15 m average depth
m3 = m2 * 15

# kg of water = m3 * 1000 kg/m3 density of water
kg_H2O = m3 * 1000

Calculate the moles of calcite needed to saturate that much water:

In [15]:
# number of moles of calcite needed to saturate all the kg of H2O in the lake
moles_calcite_for_lake = moles_calcite_per_kg_H2O * kg_H2O

moles_calcite_for_lake

76423500.00000156

Calculate the tonnage of calcite and how many hours it would take a single barge to distribute it:

In [16]:
# tons of calcite needed = moles * g/mol calcite * (1 ton / 907185 g)
tons_calcite_for_lake = moles_calcite_for_lake * 100.09 * (1 / 907185)

# hours to dispense calcite = tons of calcite * (1 hour / 10 tons)
barge_hours = tons_calcite_for_lake * (1/10)

barge_hours

843.1828254435595

So the answer is 76423500 moles of calcite to saturate the lake, which would take 843 barge-hours.

That's a lot of 9-hour shifts!